<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_Breakout_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

Run date time (IST): 2023-07-21 23:39:59


In [2]:
!pip install yfinance

In [3]:
import yfinance as yfin
#import talib
yfin.pdr_override()
#import pandas_ta as ta
import pandas as pd

In [48]:
def get_breakout(stock_symbol):

  stock_df = yfin.Ticker(stock_symbol).history(period='5y')[map(str.title, ['close'])]
  stock_df['Stock'] = stock_symbol
  stock_df = stock_df.reset_index()
  prev_max = max(stock_df[(stock_df['Date'] < '2023-01-01')]['Close'])
  tmp_df = stock_df[(stock_df['Close'] > prev_max)]
  tmp_df['DateGroup'] = tmp_df['Date'].diff().dt.days.ne(1).cumsum()
  tmp_df['Date'] = tmp_df['Date'].dt.strftime('%Y-%m-%d')
  g = tmp_df.groupby('DateGroup')['Date'].agg(['min', 'max', 'count']).reset_index()

  df_res = tmp_df.merge(g, how='left')
  return df_res[df_res['count'] > 1].head(1)

In [49]:
stock_ids = ['COLPAL.NS', 'POLYCAB.NS', 'ICICIBANK.NS']

df_breakout = pd.DataFrame()

for stock_id in stock_ids:
  tmp_df = get_breakout(stock_id)
  df_breakout = df_breakout.append(tmp_df)

df_breakout

<ipython-input-48-70b25f5d9381>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df['DateGroup'] = tmp_df['Date'].diff().dt.days.ne(1).cumsum()
<ipython-input-48-70b25f5d9381>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df['Date'] = tmp_df['Date'].dt.strftime('%Y-%m-%d')
<ipython-input-49-f684e46dbbf4>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_breakout = df_breakout.append(tmp_df)
<ipython-input-4

,Date,Close,Stock,DateGroup,min,max,count
0,2023-07-05,1793.699951,COLPAL.NS,1,2023-07-05,2023-07-07,3
1,2023-02-15,2989.476074,POLYCAB.NS,2,2023-02-15,2023-02-17,3
2,2023-07-05,958.000000,ICICIBANK.NS,3,2023-07-05,2023-07-06,2


In [ ]:
# apply for consecutive green candles...